In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
/fiap/cap14/modelos_preditivos.py

Script para desenvolvimento de modelos preditivos para recomendação
de culturas agrícolas com base em condições de solo e clima
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
import joblib

# Bibliotecas para ML
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                           f1_score, classification_report, confusion_matrix,
                           matthews_corrcoef, balanced_accuracy_score)
from sklearn.pipeline import Pipeline

# Algoritmos
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Bibliotecas para visualização
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patches as mpatches

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.1)
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

# Definir caminhos
DATA_DIR = Path('fase3_cap14/src/datasets/')
DATASET_PATH = DATA_DIR / 'Atividade_Cap_14_produtos_agricolas.csv'
OUTPUT_DIR = Path('fase3_cap14/src/datasets/output/modelos_preditivos/modelos')
OUTPUT_DIR.mkdir(exist_ok=True)
FIGURES_DIR = Path('fase3_cap14/src/datasets/output/modelos_preditivos/figuras')
FIGURES_DIR.mkdir(exist_ok=True)

# Cores personalizadas para visualizações
CUSTOM_COLORS = [
    "#4E79A7", "#F28E2B", "#E15759", "#76B7B2", "#59A14F",
    "#EDC948", "#B07AA1", "#FF9DA7", "#9C755F", "#BAB0AC"
]

# Configurar seed para reprodutibilidade
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Definir modelos a serem testados
MODELOS = {
    'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_STATE),
    'Random Forest': RandomForestClassifier(random_state=RANDOM_STATE),
    'SVM': SVC(random_state=RANDOM_STATE, probability=True),
    'KNN': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(random_state=RANDOM_STATE)
}

class AgricultureMLPipeline:
    """
    Pipeline para desenvolvimento e avaliação de modelos de ML
    para recomendação de culturas agrícolas
    """

    def __init__(self):
        """
        Inicializa o pipeline de ML
        """
        self.df = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.feature_names = None
        self.target_encoder = None
        self.models = {}
        self.results = {}
        self.best_model = None
        self.best_model_name = None

    def carregar_dados(self):
        """
        Carrega os dados do dataset
        """
        try:
            self.df = pd.read_csv(DATASET_PATH)
            print(f"Dataset carregado com sucesso: {self.df.shape[0]} linhas e {self.df.shape[1]} colunas")
            return True
        except Exception as e:
            print(f"Erro ao carregar o dataset: {e}")
            return False

    def explorar_dados(self):
        """
        Realiza exploração básica dos dados
        """
        print("\n===== Exploração dos Dados =====")

        # Informações básicas
        print("\nInformações do dataset:")
        print(self.df.info())

        print("\nEstatísticas descritivas:")
        print(self.df.describe())

        # Verificar valores ausentes
        missing_values = self.df.isnull().sum()
        if missing_values.sum() > 0:
            print("\nValores ausentes:")
            print(missing_values[missing_values > 0])
        else:
            print("\nNão foram encontrados valores ausentes no dataset.")

        # Distribuição da variável alvo
        print("\nDistribuição da variável alvo (culturas):")
        target_counts = self.df['label'].value_counts()
        print(target_counts)

        # Verificar balanceamento das classes
        print(f"\nNúmero total de classes: {len(target_counts)}")
        print(f"Classe mais frequente: {target_counts.index[0]} ({target_counts.iloc[0]} ocorrências)")
        print(f"Classe menos frequente: {target_counts.index[-1]} ({target_counts.iloc[-1]} ocorrências)")

        # Plotar distribuição das classes
        plt.figure(figsize=(10, 6))
        ax = sns.countplot(y='label', data=self.df, order=target_counts.index)
        plt.title('Distribuição das Culturas no Dataset', fontsize=14)
        plt.xlabel('Contagem', fontsize=12)
        plt.ylabel('Cultura', fontsize=12)
        # Adicionar rótulos de contagem
        for i, count in enumerate(target_counts):
            ax.text(count + 1, i, f"{count}", va='center')
        plt.tight_layout()
        plt.savefig(FIGURES_DIR / 'distribuicao_culturas.png', dpi=300)
        plt.close()

        return target_counts

    def preprocessar_dados(self, test_size=0.2, scale_features=True):
        """
        Realiza o pré-processamento dos dados e divisão em treino/teste
        """
        print("\n===== Pré-processamento dos Dados =====")

        # Separar features e target
        X = self.df.drop('label', axis=1)
        y = self.df['label']

        # Salvar nomes das features
        self.feature_names = X.columns.tolist()

        # Codificar a variável alvo
        self.target_encoder = LabelEncoder()
        y_encoded = self.target_encoder.fit_transform(y)

        # Mapear classes para facilitar a interpretação
        self.target_mapping = dict(zip(self.target_encoder.transform(self.target_encoder.classes_),
                                    self.target_encoder.classes_))

        # Dividir em conjuntos de treino e teste
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y_encoded, test_size=test_size, random_state=RANDOM_STATE, stratify=y_encoded
        )

        print(f"Dados divididos em {self.X_train.shape[0]} amostras de treino e {self.X_test.shape[0]} amostras de teste")

        # Escalar as features se solicitado
        if scale_features:
            scaler = StandardScaler()
            self.X_train = scaler.fit_transform(self.X_train)
            self.X_test = scaler.transform(self.X_test)
            print("Features normalizadas utilizando StandardScaler")

        # Verificar balanceamento nos conjuntos de treino e teste
        train_counts = np.bincount(self.y_train)
        test_counts = np.bincount(self.y_test)

        print(f"Distribuição de classes no conjunto de treino: Min={min(train_counts)}, Max={max(train_counts)}")
        print(f"Distribuição de classes no conjunto de teste: Min={min(test_counts)}, Max={max(test_counts)}")

        return True

    def treinar_modelos(self):
        """
        Treina os cinco modelos diferentes
        """
        print("\n===== Treinamento dos Modelos =====")

        self.models = {}
        self.training_times = {}

        for nome, modelo in MODELOS.items():
            print(f"\nTreinando modelo: {nome}")
            inicio = time.time()

            # Treinar o modelo
            modelo.fit(self.X_train, self.y_train)

            # Salvar modelo e tempo de treinamento
            self.models[nome] = modelo
            tempo_treino = time.time() - inicio
            self.training_times[nome] = tempo_treino

            print(f"Modelo {nome} treinado em {tempo_treino:.2f} segundos")

            # Salvar modelo em disco
            joblib.dump(modelo, OUTPUT_DIR / f"{nome.replace(' ', '_').lower()}_model.pkl")

        return self.models

    def otimizar_hiperparametros(self):
        """
        Otimiza os hiperparâmetros dos modelos utilizando GridSearchCV
        """
        print("\n===== Otimização de Hiperparâmetros =====")

        # Definir parâmetros para busca (grids)
        param_grids = {
            'Decision Tree': {
                'criterion': ['gini', 'entropy'],
                'max_depth': [None, 10, 20, 30],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4]
            },
            'Random Forest': {
                'n_estimators': [50, 100, 200],
                'max_depth': [None, 10, 20, 30],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4]
            },
            'SVM': {
                'C': [0.1, 1, 10, 100],
                'gamma': ['scale', 'auto', 0.1, 0.01],
                'kernel': ['rbf', 'linear']
            },
            'KNN': {
                'n_neighbors': [3, 5, 7, 9, 11],
                'weights': ['uniform', 'distance'],
                'p': [1, 2]  # 1 para distância Manhattan e 2 para Euclidiana
            },
            'Gradient Boosting': {
                'n_estimators': [50, 100, 200],
                'learning_rate': [0.01, 0.1, 0.2],
                'max_depth': [3, 5, 7],
                'min_samples_split': [2, 5]
            }
        }

        self.best_models = {}

        for nome, modelo in MODELOS.items():
            print(f"\nOtimizando hiperparâmetros para: {nome}")

            # Criar GridSearchCV
            grid_search = GridSearchCV(
                estimator=modelo,
                param_grid=param_grids[nome],
                cv=5,  # 5-fold cross-validation
                scoring='accuracy',
                n_jobs=-1,  # Usar todos os núcleos disponíveis
                verbose=1
            )

            # Executar busca
            inicio = time.time()
            grid_search.fit(self.X_train, self.y_train)
            tempo = time.time() - inicio

            # Salvar melhor modelo
            self.best_models[nome] = grid_search.best_estimator_

            print(f"Melhores parâmetros para {nome}: {grid_search.best_params_}")
            print(f"Melhor score de validação cruzada: {grid_search.best_score_:.4f}")
            print(f"Tempo de otimização: {tempo:.2f} segundos")

            # Salvar melhor modelo em disco
            joblib.dump(grid_search.best_estimator_,
                      OUTPUT_DIR / f"{nome.replace(' ', '_').lower()}_best_model.pkl")

        return self.best_models

    def avaliar_modelos(self, use_best_models=True):
        """
        Avalia o desempenho dos modelos no conjunto de teste
        """
        print("\n===== Avaliação dos Modelos =====")

        self.results = {}

        # Escolher quais modelos avaliar (básicos ou otimizados)
        modelos_avaliar = self.best_models if use_best_models else self.models

        for nome, modelo in modelos_avaliar.items():
            print(f"\nAvaliando modelo: {nome}")

            # Fazer previsões
            inicio = time.time()
            y_pred = modelo.predict(self.X_test)
            tempo_inferencia = time.time() - inicio

            # Calcular métricas
            acc = accuracy_score(self.y_test, y_pred)
            balanced_acc = balanced_accuracy_score(self.y_test, y_pred)
            precision = precision_score(self.y_test, y_pred, average='weighted')
            recall = recall_score(self.y_test, y_pred, average='weighted')
            f1 = f1_score(self.y_test, y_pred, average='weighted')
            mcc = matthews_corrcoef(self.y_test, y_pred)

            # Armazenar resultados
            self.results[nome] = {
                'accuracy': acc,
                'balanced_accuracy': balanced_acc,
                'precision': precision,
                'recall': recall,
                'f1_score': f1,
                'matthews_corrcoef': mcc,
                'inference_time': tempo_inferencia
            }

            # Imprimir resultados principais
            print(f"Acurácia: {acc:.4f}")
            print(f"Acurácia Balanceada: {balanced_acc:.4f}")
            print(f"Precisão: {precision:.4f}")
            print(f"Recall: {recall:.4f}")
            print(f"F1-Score: {f1:.4f}")
            print(f"Coeficiente de Matthews: {mcc:.4f}")
            print(f"Tempo de inferência: {tempo_inferencia:.6f} segundos")

            # Gerar relatório de classificação detalhado
            print("\nRelatório de classificação detalhado:")
            class_report = classification_report(
                self.y_test,
                y_pred,
                target_names=self.target_encoder.classes_
            )
            print(class_report)

            # Gerar e salvar matriz de confusão
            self._gerar_matriz_confusao(
                self.y_test,
                y_pred,
                nome,
                self.target_encoder.classes_
            )

        # Identificar o melhor modelo
        self._identificar_melhor_modelo()

        return self.results

    def _identificar_melhor_modelo(self):
        """
        Identifica o melhor modelo com base no F1-Score
        """
        # Ordenar modelos por F1-Score
        modelos_ordenados = sorted(
            self.results.items(),
            key=lambda x: x[1]['f1_score'],
            reverse=True
        )

        self.best_model_name = modelos_ordenados[0][0]
        self.best_model = self.best_models[self.best_model_name]

        print(f"\nMelhor modelo: {self.best_model_name}")
        print(f"F1-Score: {self.results[self.best_model_name]['f1_score']:.4f}")

        return self.best_model_name

    def _gerar_matriz_confusao(self, y_true, y_pred, model_name, class_names):
        """
        Gera e salva a matriz de confusão para um modelo
        """
        # Calcular matriz de confusão
        cm = confusion_matrix(y_true, y_pred)

        # Normalizar matriz (opcional para melhor visualização)
        cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        # Criar figura
        plt.figure(figsize=(12, 10))

        # Plotar matriz de confusão
        sns.heatmap(
            cm_norm,
            annot=True,
            fmt=".2f",
            cmap="Blues",
            xticklabels=class_names,
            yticklabels=class_names
        )

        plt.title(f'Matriz de Confusão Normalizada - {model_name}', fontsize=16)
        plt.ylabel('Classe Real', fontsize=14)
        plt.xlabel('Classe Prevista', fontsize=14)
        plt.tight_layout()

        # Salvar figura
        plt.savefig(
            FIGURES_DIR / f'confusion_matrix_{model_name.replace(" ", "_").lower()}.png',
            dpi=300
        )
        plt.close()

    def comparar_modelos(self):
        """
        Compara o desempenho dos modelos visualmente
        """
        print("\n===== Comparação de Modelos =====")

        # Preparar dados para visualização
        modelos = list(self.results.keys())
        metricas = ['accuracy', 'balanced_accuracy', 'precision',
                   'recall', 'f1_score', 'matthews_corrcoef']

        # Criar DataFrame com resultados
        resultados_df = pd.DataFrame({
            'Modelo': [],
            'Métrica': [],
            'Valor': []
        })

        for modelo in modelos:
            for metrica in metricas:
                resultados_df = pd.concat([resultados_df, pd.DataFrame({
                    'Modelo': [modelo],
                    'Métrica': [metrica],
                    'Valor': [self.results[modelo][metrica]]
                })], ignore_index=True)

        # Criar gráfico de barras para comparação
        plt.figure(figsize=(14, 10))
        sns.barplot(
            data=resultados_df,
            x='Métrica',
            y='Valor',
            hue='Modelo',
            palette=CUSTOM_COLORS[:len(modelos)]
        )

        plt.title('Comparação de Performance entre Modelos', fontsize=16)
        plt.xlabel('Métrica', fontsize=14)
        plt.ylabel('Valor', fontsize=14)
        plt.ylim(0, 1)
        plt.xticks(rotation=45)
        plt.legend(title='Modelo', loc='lower right')
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()

        plt.savefig(FIGURES_DIR / 'comparacao_modelos.png', dpi=300)
        plt.close()

        # Criar tabela de resultados
        tabela_resultados = pd.DataFrame()

        for modelo in modelos:
            tabela_resultados[modelo] = pd.Series({
                'Acurácia': self.results[modelo]['accuracy'],
                'Acurácia Balanceada': self.results[modelo]['balanced_accuracy'],
                'Precisão': self.results[modelo]['precision'],
                'Recall': self.results[modelo]['recall'],
                'F1-Score': self.results[modelo]['f1_score'],
                'Coef. Matthews': self.results[modelo]['matthews_corrcoef'],
                'Tempo Inferência (s)': self.results[modelo]['inference_time']
            })

        # Salvar tabela
        tabela_resultados.to_csv(OUTPUT_DIR / 'comparacao_modelos.csv')

        print("\nTabela comparativa de modelos:")
        print(tabela_resultados)

        return tabela_resultados

    def analisar_importancia_features(self):
        """
        Analisa a importância das features para os modelos que suportam
        essa funcionalidade (Decision Tree, Random Forest, Gradient Boosting)
        """
        print("\n===== Análise de Importância de Features =====")

        # Modelos que suportam importância de features
        modelos_suportados = [
            'Decision Tree',
            'Random Forest',
            'Gradient Boosting'
        ]

        importancias = {}

        # Coletar importância de features dos modelos
        for nome in modelos_suportados:
            if nome in self.best_models:
                modelo = self.best_models[nome]

                # Extrair importância de features
                if hasattr(modelo, 'feature_importances_'):
                    importancias[nome] = dict(zip(
                        self.feature_names,
                        modelo.feature_importances_
                    ))

                    # Ordenar features por importância
                    importancias[nome] = dict(sorted(
                        importancias[nome].items(),
                        key=lambda x: x[1],
                        reverse=True
                    ))

                    print(f"\nImportância de features para {nome}:")
                    for feature, imp in importancias[nome].items():
                        print(f"  {feature}: {imp:.4f}")

        # Visualizar importância de features
        if importancias:
            self._visualizar_importancia_features(importancias)

        return importancias

    def _visualizar_importancia_features(self, importancias):
        """
        Cria visualizações para importância de features
        """
        for nome, imp in importancias.items():
            # Converter para DataFrame e ordenar
            df_imp = pd.DataFrame({
                'Feature': list(imp.keys()),
                'Importância': list(imp.values())
            }).sort_values('Importância', ascending=False)

            # Plotar gráfico de barras horizontais
            plt.figure(figsize=(10, 8))
            sns.barplot(
                data=df_imp,
                y='Feature',
                x='Importância',
                palette='viridis'
            )

            plt.title(f'Importância de Features - {nome}', fontsize=14)
            plt.xlabel('Importância', fontsize=12)
            plt.ylabel('Feature', fontsize=12)
            plt.tight_layout()

            plt.savefig(
                FIGURES_DIR / f'feature_importance_{nome.replace(" ", "_").lower()}.png',
                dpi=300
            )
            plt.close()

    def extrair_regras_decision_tree(self):
        """
        Extrai regras interpretáveis da Decision Tree
        """
        if 'Decision Tree' not in self.best_models:
            print("Modelo Decision Tree não disponível para extração de regras.")
            return None

        print("\n===== Extração de Regras da Decision Tree =====")

        tree_model = self.best_models['Decision Tree']

        # Extrair regras da árvore usando o método auxiliar
        from sklearn.tree import export_text

        regras = export_text(
            tree_model,
            feature_names=self.feature_names,
            max_depth=3,  # Limitar profundidade para facilitar interpretação
            decimals=2
        )

        print("\nRegras extraídas da Decision Tree (limitadas à profundidade 3):")
        print(regras)

        # Salvar regras em um arquivo
        with open(OUTPUT_DIR / 'decision_tree_rules.txt', 'w') as f:
            f.write(regras)

        return regras

    def gerar_exemplos_praticos(self):
        """
        Gera exemplos práticos de previsões usando o melhor modelo
        """
        print("\n===== Exemplos Práticos de Previsões =====")

        if not self.best_model:
            print("Nenhum modelo identificado como o melhor.")
            return

        # Selecionar alguns exemplos reais do conjunto de teste
        indices = np.random.choice(len(self.X_test), 5, replace=False)
        X_exemplos = self.X_test[indices]
        y_real = self.y_test[indices]

        # Fazer previsões com o melhor modelo
        y_pred = self.best_model.predict(X_exemplos)

        # Se o modelo suporta probabilidades, extrair também
        if hasattr(self.best_model, 'predict_proba'):
            y_proba = self.best_model.predict_proba(X_exemplos)
            probabilidades = np.max(y_proba, axis=1)
        else:
            probabilidades = [None] * len(y_pred)

        # Converter índices previstos para nomes de culturas
        culturas_reais = [self.target_mapping[y] for y in y_real]
        culturas_previstas = [self.target_mapping[y] for y in y_pred]

        # Mostrar resultados
        print(f"\nExemplos de previsões utilizando o modelo {self.best_model_name}:")

        for i in range(len(X_exemplos)):
            print(f"\nExemplo {i+1}:")

            # Converter vetor de features para dicionário com nomes
            if isinstance(X_exemplos[i], np.ndarray):
                features = dict(zip(self.feature_names, X_exemplos[i]))
            else:
                features = X_exemplos[i]

            for feat, val in features.items():
                print(f"  {feat}: {val:.2f}")

            print(f"  Cultura real: {culturas_reais[i]}")
            print(f"  Cultura prevista: {culturas_previstas[i]}")

            if probabilidades[i] is not None:
                print(f"  Confiança da previsão: {probabilidades[i]:.2f}")

            # Indicar se a previsão está correta
            print(f"  Previsão correta: {'Sim' if culturas_reais[i] == culturas_previstas[i] else 'Não'}")

        return {'X_exemplos': X_exemplos, 'y_real': culturas_reais,
               'y_pred': culturas_previstas, 'proba': probabilidades}

    def executar_pipeline_completo(self):
        """
        Executa o pipeline completo de ML
        """
        print("\n===== Iniciando Pipeline Completo de ML =====")

        # Etapa 1: Carregar dados
        if not self.carregar_dados():
            return "Falha ao carregar os dados. Pipeline interrompido."

        # Etapa 2: Explorar dados
        self.explorar_dados()

        # Etapa 3: Pré-processar dados
        self.preprocessar_dados()

        # Etapa 4: Treinar modelos básicos
        self.treinar_modelos()

        # Etapa 5: Otimizar hiperparâmetros
        self.otimizar_hiperparametros()

        # Etapa 6: Avaliar modelos otimizados
        self.avaliar_modelos(use_best_models=True)

        # Etapa 7: Comparar modelos
        self.comparar_modelos()

        # Etapa 8: Analisar importância de features
        self.analisar_importancia_features()

        # Etapa 9: Extrair regras da Decision Tree
        self.extrair_regras_decision_tree()

        # Etapa 10: Gerar exemplos práticos
        self.gerar_exemplos_praticos()

        print("\n===== Pipeline Completo Finalizado =====")
        print(f"Todos os modelos e resultados foram salvos no diretório: {OUTPUT_DIR}")
        print(f"Todas as visualizações foram salvas no diretório: {FIGURES_DIR}")

        # Retornar informações sobre o melhor modelo
        return {
            'best_model_name': self.best_model_name,
            'best_model_performance': self.results[self.best_model_name],
            'feature_names': self.feature_names,
            'target_mapping': self.target_mapping
        }

def criar_app_predicao(pipeline):
    """
    Cria um aplicativo simples para previsão interativa
    """
    print("\n===== Criando Aplicativo de Previsão =====")

    # Extrair informações necessárias
    if not pipeline.best_model:
        print("Nenhum modelo treinado disponível.")
        return

    # Verificar se pipeline possui informações necessárias
    if not hasattr(pipeline, 'feature_names') or not hasattr(pipeline, 'target_mapping'):
        print("Informações necessárias não encontradas no pipeline.")
        return

    # Criar arquivo Python com aplicativo de previsão
    app_code = f"""#!/usr/bin/env python3
# -*- coding: utf-8 -*-
'''
/fiap/cap14/app_predicao.py

Aplicativo simples para prever a cultura ideal com base em condições de solo e clima
'''

import joblib
import numpy as np
import argparse
import sys
from pathlib import Path

# Carregar modelo
MODEL_PATH = Path('{OUTPUT_DIR}/{pipeline.best_model_name.replace(" ", "_").lower()}_best_model.pkl')
modelo = joblib.load(MODEL_PATH)

# Definir nomes das features e mapeamento de classes
feature_names = {pipeline.feature_names}
target_mapping = {pipeline.target_mapping}

def prever_cultura(valores):
    '''
    Prevê a cultura ideal com base nos valores informados

    Parâmetros:
    valores (list): Lista com os valores na ordem: {', '.join([str(f) for f in pipeline.feature_names])}

    Retorna:
    dict: Dicionário com a cultura prevista e confiança (se disponível)
    '''
    # Converter entrada para array numpy
    X = np.array(valores).reshape(1, -1)

    # Fazer previsão
    y_pred = modelo.predict(X)[0]
    cultura = target_mapping.get(y_pred, "Desconhecida")

    # Tentar obter probabilidades (se o modelo suportar)
    confianca = None
    try:
        if hasattr(modelo, 'predict_proba'):
            proba = modelo.predict_proba(X)
            confianca = float(np.max(proba))
    except:
        pass

    return {{
        'cultura': cultura,
        'confianca': confianca
    }}

def exemplo_interativo():
    '''
    Executa um exemplo interativo de previsão
    '''
    print("\\n===== Sistema de Previsão de Cultura Ideal =====")
    print(f"Utilizando o modelo: {{MODEL_PATH.name}}")
    print("\\nDigite os valores para as seguintes condições:")

    valores = []

    for feature in feature_names:
        while True:
            try:
                valor = float(input(f"{{feature}}: "))
                valores.append(valor)
                break
            except ValueError:
                print("Por favor, digite um valor numérico válido.")

    resultado = prever_cultura(valores)

    print("\\n----- Resultado -----")
    print(f"Cultura ideal prevista: {{resultado['cultura']}}")

    if resultado['confianca']:
        print(f"Confiança da previsão: {{resultado['confianca']:.2f}}")

    print("\\nObservação: Este é um modelo experimental e os resultados devem ser")
    print("validados por especialistas em agricultura antes de aplicação prática.")

def modo_nao_interativo(valores):
    '''
    Executa o modelo em modo não-interativo

    Parâmetros:
    valores (list): Lista com os valores na ordem: {', '.join([str(f) for f in pipeline.feature_names])}
    '''
    if len(valores) != len(feature_names):
        print(f"Erro: Número incorreto de valores. Esperado {{len(feature_names)}} valores.")
        print(f"Ordem dos valores: {{', '.join(feature_names)}}")
        sys.exit(1)

    try:
        # Converter para float
        valores_float = [float(v) for v in valores]

        # Fazer previsão
        resultado = prever_cultura(valores_float)

        print("\\n----- Resultado -----")
        print(f"Cultura ideal prevista: {{resultado['cultura']}}")

        if resultado['confianca']:
            print(f"Confiança da previsão: {{resultado['confianca']:.2f}}")

        return resultado
    except ValueError as e:
        print(f"Erro ao converter valores: {{e}}")
        sys.exit(1)

def usar_exemplo_real():
    '''
    Usa um exemplo real do conjunto de dados para demonstração
    '''
    # Carregar dataset original
    try:
        import pandas as pd
        DATA_DIR = Path('fase3_cap14/src/datasets/')
        DATASET_PATH = DATA_DIR / 'Atividade_Cap_14_produtos_agricolas.csv'
        df = pd.read_csv(DATASET_PATH)

        # Selecionar uma amostra aleatória
        amostra = df.sample(n=1, random_state=42)

        # Extrair valores das features
        valores = amostra.drop('label', axis=1).values[0].tolist()
        cultura_real = amostra['label'].values[0]

        # Fazer previsão
        resultado = prever_cultura(valores)

        print("\\n===== Exemplo com Dados Reais =====")
        print("\\nValores das características:")
        for i, feat in enumerate(feature_names):
            print(f"  {{feat}}: {{valores[i]:.2f}}")

        print("\\n----- Resultado -----")
        print(f"Cultura real: {{cultura_real}}")
        print(f"Cultura prevista: {{resultado['cultura']}}")

        if resultado['confianca']:
            print(f"Confiança da previsão: {{resultado['confianca']:.2f}}")

        print(f"Previsão correta: {{cultura_real == resultado['cultura']}}")

        return resultado
    except Exception as e:
        print(f"Erro ao usar exemplo real: {{e}}")
        return None


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Previsão de cultura agrícola ideal')
    parser.add_argument('--valores', nargs='+', help=f"Valores das características na ordem: {{', '.join(feature_names)}}")
    parser.add_argument('--exemplo-real', action='store_true', help='Usar um exemplo real do conjunto de dados')

    args = parser.parse_args()

    if args.exemplo_real:
        # Usar exemplo real do dataset
        usar_exemplo_real()
    elif args.valores:
        # Modo não-interativo com valores fornecidos
        modo_nao_interativo(args.valores)
    else:
        # Modo interativo
        exemplo_interativo()
"""

    # Salvar o aplicativo
    app_path = (Path(DATA_DIR) / 'output' / 'modelos_preditivos' / 'utils' / 'app_predicao.py').resolve()
    app_path.parent.mkdir(parents=True, exist_ok=True)
    with open(app_path, 'w') as f:
        f.write(app_code)

    print(f"Aplicativo de previsão criado em: {app_path}")
    print("Para usar o aplicativo, execute para obter informações: python app_predicao.py -h")

    return app_path

def main():
    """
    Função principal para execução do pipeline ML
    """
    print("===== Iniciando Desenvolvimento de Modelos Preditivos =====")

    # Criar e executar pipeline
    pipeline = AgricultureMLPipeline()
    resultados = pipeline.executar_pipeline_completo()

    # Criar aplicativo de previsão
    criar_app_predicao(pipeline)

    return resultados

if __name__ == "__main__":
    main()


===== Iniciando Desenvolvimento de Modelos Preditivos =====

===== Iniciando Pipeline Completo de ML =====
Dataset carregado com sucesso: 2200 linhas e 8 colunas

===== Exploração dos Dados =====

Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            2200 non-null   int64  
 1   P            2200 non-null   int64  
 2   K            2200 non-null   int64  
 3   temperature  2200 non-null   float64
 4   humidity     2200 non-null   float64
 5   ph           2200 non-null   float64
 6   rainfall     2200 non-null   float64
 7   label        2200 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 137.6+ KB
None

Estatísticas descritivas:
                 N            P            K  temperature     humidity  \
count  2200.000000  2200.000000  2200.000000  2200.000000  2200.000000   
mean     

In [64]:
!python3 fase3_cap14/src/datasets/output/modelos_preditivos/utils/app_predicao.py --valores 90 42 43 20.87 82.18 6.5 202.9


----- Resultado -----
Cultura ideal prevista: apple
Confiança da previsão: 0.42


In [70]:
!python3 fase3_cap14/src/datasets/output/modelos_preditivos/utils/app_predicao.py -h

usage: app_predicao.py [-h] [--valores VALORES [VALORES ...]] [--exemplo-real]

Previsão de cultura agrícola ideal

options:
  -h, --help            show this help message and exit
  --valores VALORES [VALORES ...]
                        Valores das características na ordem: N, P, K,
                        temperature, humidity, ph, rainfall
  --exemplo-real        Usar um exemplo real do conjunto de dados


In [73]:
!python3 fase3_cap14/src/datasets/output/modelos_preditivos/utils/app_predicao.py --exemplo-real


===== Exemplo com Dados Reais =====

Valores das características:
  N: 101.00
  P: 17.00
  K: 47.00
  temperature: 29.49
  humidity: 94.73
  ph: 6.19
  rainfall: 26.31

----- Resultado -----
Cultura real: muskmelon
Cultura prevista: apple
Confiança da previsão: 0.42
Previsão correta: False


In [55]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
'''
ADAPTADO PARA EXECUÇÃO EM CÉLULA

/fiap/cap14/app_predicao.py
Aplicativo simples para prever a cultura ideal com base em condições de solo e clima
'''

import joblib
import numpy as np
import argparse
import sys
from pathlib import Path

# Carregar modelo
MODEL_PATH = Path('fase3_cap14/src/datasets/output/modelos_preditivos/modelos/random_forest_best_model.pkl')
modelo = joblib.load(MODEL_PATH)

# Definir nomes das features e mapeamento de classes
feature_names = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']
target_mapping = {0: 'apple', 1: 'banana', 2: 'blackgram', 3: 'chickpea', 4: 'coconut', 5: 'coffee', 6: 'cotton', 7: 'grapes', 8: 'jute', 9: 'kidneybeans', 10: 'lentil', 11: 'maize', 12: 'mango', 13: 'mothbeans', 14: 'mungbean', 15: 'muskmelon', 16: 'orange', 17: 'papaya', 18: 'pigeonpeas', 19: 'pomegranate', 20: 'rice', 21: 'watermelon'}

def prever_cultura(valores):
    '''
    Prevê a cultura ideal com base nos valores informados

    Parâmetros:
    valores (list): Lista com os valores na ordem: N, P, K, temperature, humidity, ph, rainfall

    Retorna:
    dict: Dicionário com a cultura prevista e confiança (se disponível)
    '''
    # Converter entrada para array numpy
    X = np.array(valores).reshape(1, -1)

    # Fazer previsão
    y_pred = modelo.predict(X)[0]
    cultura = target_mapping.get(y_pred, "Desconhecida")

    # Tentar obter probabilidades (se o modelo suportar)
    confianca = None
    try:
        if hasattr(modelo, 'predict_proba'):
            proba = modelo.predict_proba(X)
            confianca = float(np.max(proba))
    except:
        pass

    return {
        'cultura': cultura,
        'confianca': confianca
    }

def exemplo_interativo():
    '''
    Executa um exemplo interativo de previsão
    '''
    print("\n===== Sistema de Previsão de Cultura Ideal =====")
    print(f"Utilizando o modelo: {MODEL_PATH.name}")
    print("\nDigite os valores para as seguintes condições:")

    valores = []

    for feature in feature_names:
        while True:
            try:
                valor = float(input(f"{feature}: "))
                valores.append(valor)
                break
            except ValueError:
                print("Por favor, digite um valor numérico válido.")

    resultado = prever_cultura(valores)

    print("\n----- Resultado -----")
    print(f"Cultura ideal prevista: {resultado['cultura']}")

    if resultado['confianca']:
        print(f"Confiança da previsão: {resultado['confianca']:.2f}")

    print("\nObservação: Este é um modelo experimental e os resultados devem ser")
    print("validados por especialistas em agricultura antes de aplicação prática.")

def modo_nao_interativo(valores):
    '''
    Executa o modelo em modo não-interativo

    Parâmetros:
    valores (list): Lista com os valores na ordem: N, P, K, temperature, humidity, ph, rainfall
    '''
    if len(valores) != len(feature_names):
        print(f"Erro: Número incorreto de valores. Esperado {len(feature_names)} valores.")
        print(f"Ordem dos valores: {', '.join(feature_names)}")
        sys.exit(1)

    try:
        # Converter para float
        valores_float = [float(v) for v in valores]

        # Fazer previsão
        resultado = prever_cultura(valores_float)

        print("\n----- Resultado -----")
        print(f"Cultura ideal prevista: {resultado['cultura']}")

        if resultado['confianca']:
            print(f"Confiança da previsão: {resultado['confianca']:.2f}")

        return resultado
    except ValueError as e:
        print(f"Erro ao converter valores: {e}")
        sys.exit(1)

def usar_exemplo_real():
    '''
    Usa um exemplo real do conjunto de dados para demonstração
    '''
    # Carregar dataset original
    try:
        import pandas as pd
        DATA_DIR = Path('fase3_cap14/src/datasets/')
        DATASET_PATH = DATA_DIR / 'Atividade_Cap_14_produtos_agricolas.csv'
        df = pd.read_csv(DATASET_PATH)

        # Selecionar uma amostra aleatória
        amostra = df.sample(n=1, random_state=42)

        # Extrair valores das features
        valores = amostra.drop('label', axis=1).values[0].tolist()
        cultura_real = amostra['label'].values[0]

        # Fazer previsão
        resultado = prever_cultura(valores)

        print("\n===== Exemplo com Dados Reais =====")
        print("\nValores das características:")
        for i, feat in enumerate(feature_names):
            print(f"  {feat}: {valores[i]:.2f}")

        print("\n----- Resultado -----")
        print(f"Cultura real: {cultura_real}")
        print(f"Cultura prevista: {resultado['cultura']}")

        if resultado['confianca']:
            print(f"Confiança da previsão: {resultado['confianca']:.2f}")

        print(f"Previsão correta: {cultura_real == resultado['cultura']}")

        return resultado
    except Exception as e:
        print(f"Erro ao usar exemplo real: {e}")
        return None

if __name__ == "__main__":
    usar_exemplo_real()



===== Exemplo com Dados Reais =====

Valores das características:
  N: 101.00
  P: 17.00
  K: 47.00
  temperature: 29.49
  humidity: 94.73
  ph: 6.19
  rainfall: 26.31

----- Resultado -----
Cultura real: muskmelon
Cultura prevista: apple
Confiança da previsão: 0.42
Previsão correta: False


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b5dc9755-22da-4b0a-a5aa-ce1cc97fc743' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>